In [3]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# pd.options.display.max_columns = None
# pd.options.display.max_rows = None

In [4]:
ticker_names = pd.read_csv(os.path.join('Resources', 'other-listed.csv')).\
rename(columns={'ACT Symbol':'Ticker'}).\
drop(columns=['Security Name', 'Exchange', 'CQS Symbol',
       'ETF', 'Round Lot Size', 'Test Issue', 'NASDAQ Symbol'])

In [5]:
data_2018 = pd.read_csv(os.path.join('Resources', '2018_Financial_Data.csv')).\
rename(columns={'Unnamed: 0':'Ticker'}).\
sort_values(by='Ticker').\
reset_index(drop=True)

In [6]:
nyse = pd.read_csv(os.path.join('Resources', 'nyse.csv'))
nasdaq = pd.read_csv(os.path.join('Resources', 'nasdaq.csv'))
frames = [nyse, nasdaq]

In [7]:
tickers = pd.concat(frames).sort_values(by='Ticker')

In [8]:
df = tickers.merge(data_2018, on='Ticker')

In [9]:
df.head();

In [31]:
df.iloc[:,0:40];

In [32]:
df.iloc[:,40:80];

In [ ]:
# df[df['Ticker'] == "GPS"]

In [ ]:
# df.to_csv(os.path.join('2018_data.csv'))